# 라이브러리 설치

In [2]:
pip install httplib2

   ---------------------------------------- 0.0/96.9 kB ? eta -:--:--
   ------------ --------------------------- 30.7/96.9 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 96.9/96.9 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/103.2 kB ? eta -:--:--
   ---------------------------------------- 103.2/103.2 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install google-api-python-client

   ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
    --------------------------------------- 0.2/12.5 MB 5.1 MB/s eta 0:00:03
   - -------------------------------------- 0.4/12.5 MB 4.4 MB/s eta 0:00:03
   - -------------------------------------- 0.6/12.5 MB 4.6 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/12.5 MB 4.6 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/12.5 MB 4.7 MB/s eta 0:00:03
   --- ------------------------------------ 1.1/12.5 MB 4.6 MB/s eta 0:00:03
   --- ------------------------------------ 1.2/12.5 MB 4.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.4/12.5 MB 3.8 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/12.5 MB 3.9 MB/s eta 0:00:03
   ----- ---------------------------------- 1.7/12.5 MB 3.8 MB/s eta 0:00:03
   ------ --------------------------------- 1.9/12.5 MB 3.9 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.5 MB 3.8 MB/s eta 0:00:03
   ---

In [7]:
pip install oauth2client

   ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
   ------------ --------------------------- 30.7/98.2 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 98.2/98.2 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# GSC API 호출

In [1]:
import argparse
import httplib2
import requests
import pandas as pd 

from collections import defaultdict
from dateutil import relativedelta
from googleapiclient.discovery import build
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [2]:
# Google Cloud Project Client ID & Client Secrets
CLIENT_ID = "165277451909-549djem84js33jh3thql01iccovj6hsl.apps.googleusercontent.com"
CLIENT_SECRET = "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"

# Define Oath scopes with read only access
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"

parser = argparse.ArgumentParser(
    formatter_class=argparse.RawDescriptionHelpFormatter,
    parents=[tools.argparser])
flags = parser.parse_args([])
 
flow = client.OAuth2WebServerFlow(client_id=CLIENT_ID,
                           client_secret=CLIENT_SECRET,
                           scope=OAUTH_SCOPE)
 
storage = file.Storage('searchconsole.dat')
 
credentials = storage.get()
if credentials is None or credentials.invalid:
  credentials = tools.run_flow(flow, storage, flags)
 
# Create an httplib2.Http object and authorize it with our credentials
http = credentials.authorize(http=httplib2.Http())

In [3]:
webmasters_service = build('searchconsole', 'v1', http)
webmasters_service

# Page, Query by month


In [16]:
import pandas as pd 

# the website we want to get the data for
websites = ["https://www.samsung.com/us/", "https://www.samsung.com/uk/", "https://www.samsung.com/in/", "https://www.samsung.com/au/", "https://www.samsung.com/ca/", "https://www.samsung.com/sec/", "https://www.samsung.com/sg/", "https://www.samsung.com/jp/"]
startdate = ['2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01']
enddate = ['2022-12-31', '2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29', '2024-03-31']

for property in websites:
    for i in range(len(startdate)):
        all_response = []  # Initialize all_response for each website
        
        startRow = 0

        while (startRow == 0) or (startRow % 25000 == 0):
            request_body = {
                "startDate": startdate[i],
                "endDate": enddate[i],
                "dimensions": ['Page', 'Query'],
                "rowLimit": 25000,
                "dataState": "final",
                "startRow": startRow
            }
            response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
            startRow += len(response_data['rows'])
            print("fetched up to " + str(startRow) + " rows of data")

            all_response.extend(response_data['rows'])

        data_for_df = []

        for row in all_response:
            temp = []
            temp.append(startdate[i])    #month
            temp.append(row['keys'][0])  #page
            temp.append(row['keys'][1])  #query
            temp.append(row['clicks'])    # clicks
            temp.append(row['impressions'])  # impressions
            temp.append(row['ctr'])       # ctr
            temp.append(row['position'])  # position
            temp.append(property)         # website
            data_for_df.append(temp)

        df = pd.DataFrame(data_for_df, columns=['month', 'page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'website'])
        df.to_csv(f'{property.split("/")[-2]}-{startdate[i]}-month-page-query.csv', index=False)

fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 200000 rows of data
fetched up to 225000 rows of data
fetched up to 234450 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 200000 rows of data
fetched up to 225000 rows of data
fetched up to 234489 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 200000 rows of data
fetched up to 206619 rows of data
fetched up to 25000 row

KeyError: 'rows'

## jp용

In [6]:
import pandas as pd 

# the website we want to get the data for
websites = ["https://www.samsung.com/jp/"]
startdate = ['2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01']
enddate = ['2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29', '2024-03-31']

for property in websites:
    for i in range(len(startdate)):
        all_response = []  # Initialize all_response for each website
        
        startRow = 0

        while (startRow == 0) or (startRow % 25000 == 0):
            request_body = {
                "startDate": startdate[i],
                "endDate": enddate[i],
                "dimensions": ['Page', 'Query'],
                "rowLimit": 25000,
                "dataState": "final",
                "startRow": startRow
            }
            response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
            startRow += len(response_data['rows'])
            print("fetched up to " + str(startRow) + " rows of data")

            all_response.extend(response_data['rows'])

        data_for_df = []

        for row in all_response:
            temp = []
            temp.append(startdate[i])    #month
            temp.append(row['keys'][0])  #page
            temp.append(row['keys'][1])  #query
            temp.append(row['clicks'])    # clicks
            temp.append(row['impressions'])  # impressions
            temp.append(row['ctr'])       # ctr
            temp.append(row['position'])  # position
            temp.append(property)         # website
            data_for_df.append(temp)

        df = pd.DataFrame(data_for_df, columns=['month', 'page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'website'])
        df.to_csv(f'{property.split("/")[-2]}-{startdate[i]}-month-page-query.csv', index=False)

fetched up to 25000 rows of data
fetched up to 35696 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 200000 rows of data
fetched up to 200737 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 182219 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 183302 rows of data
fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of

# 언제부터 뽑히는지 확인

In [4]:
import pandas as pd 

# 테스트하고자 하는 국가
websites = ["https://www.samsung.com/jp/", "https://www.samsung.com/in/"]
# 시작일과 종료일
startdate = '2022-11-01'
enddate = '2023-03-31'

for property in websites:
    all_response = []  # 각 웹사이트별로 all_response를 초기화합니다.
    startRow = 0

    while (startRow == 0) or (startRow % 25000 == 0):
        request_body = {
            "startDate": startdate,
            "endDate": enddate,
            "dimensions": ['Date'],
            "rowLimit": 25000,
            "dataState": "final",
            "startRow": startRow
        }
        response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
        startRow += len(response_data['rows'])
        print("fetched up to " + str(startRow) + " rows of data")

        all_response.extend(response_data['rows'])

    if all_response:
        print(f"Data for {property} starts from {all_response[0]['keys'][0]}")
    else:
        print(f"No data found for {property} within the specified date range.")


fetched up to 38 rows of data
Data for https://www.samsung.com/jp/ starts from 2023-02-22
fetched up to 120 rows of data
Data for https://www.samsung.com/in/ starts from 2022-12-02
